In [1]:
!pip install keras-tuner

  Using cached keras_tuner-1.1.2-py3-none-any.whl (133 kB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_available

True

In [8]:
fashion_mnist = keras.datasets.fashion_mnist

In [9]:
(train_img, train_lbls), (test_img, test_lbl) = fashion_mnist.load_data()

In [10]:
# scaling
train_img = train_img/255
test_img = test_img/255

In [11]:
train_img[0].shape

(28, 28)

In [12]:
train_img = train_img.reshape(len(train_img), 28, 28, 1)
test_img = test_img.reshape(len(test_img), 28, 28, 1)

In [14]:
test_img[0].shape

(28, 28, 1)

In [19]:
def build_model(hp):
    
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [20]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\Visha\AppData\Local\Temp\ipykernel_17004\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [21]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [25]:
tuner_search.search(train_img,train_lbls,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.906499981880188

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 00h 02m 21s
INFO:tensorflow:Oracle triggered exit


In [26]:
model=tuner_search.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        179264    
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 64)                1982528   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,163,562
Trainable params: 2,163,562
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(train_img, train_lbls, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1352 - accuracy: 0.9484 - val_loss: 0.2901 - val_accuracy: 0.9098
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0968 - accuracy: 0.9639 - val_loss: 0.2942 - val_accuracy: 0.9090
Epoch 6/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0684 - accuracy: 0.9749 - val_loss: 0.3206 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0476 - accuracy: 0.9823 - val_loss: 0.3805 - val_accuracy: 0.9073
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0424 - accuracy: 0.9847 - val_loss: 0.4481 - val_accuracy: 0.9080
Epoch 9/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0304 - accuracy: 0.9891 - val_loss: 0.4555 - val_accuracy: 0.9103
Epoch 10/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0268 - accuracy: 0.9902 - val_loss: 0.5206 - val_accur